<a href="https://colab.research.google.com/github/nermienkh/POC-Text-2-Text-Chatbot/blob/main/TyDiQA_Subtask2_Arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#install needed dependencies

In [1]:
!nvidia-smi

Mon Jul  3 12:51:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install datasets
!pip install transformers==4.2.16
!pip install sentencepiece
#only needed for AraBERTv1 and v2
!pip install farasapy
!pip install pyarabic
!pip install fuzzysearch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.9 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement transformers==4.2.16 (from versions: 0.1, 2.0.0, 2.1.0, 2.1.1, 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.8.0, 2.9.0, 2.9.1, 2.10.0, 2.11.0, 3.0.0, 3.0.1, 3.0.2, 3.1.0, 3.2.0, 3.3.0, 3.3.1, 3.4.0, 3.5.0, 3.5.1, 4.0.0rc1, 4.0.0, 4.0.1, 4.1.0, 4.1.1, 4.2.0, 4.2.1, 4.2.2, 4.3.0rc1, 4.3.0, 4.3.1, 4.3.2, 4.3.3, 4.4.0, 4.4.1, 4.4.2, 4.5.0, 4.5.1, 4.6.0, 4.6.1, 4.7.0, 4.8.0, 4.8.1, 4.8.2, 4.9.0, 4.9.1, 4.9.2, 4.10.0, 4.10.1, 4.10.2, 4.10.3, 4.11.0, 4.11.1, 4.11.2, 4.11.3, 4.12.0, 4.12.1, 4.12.2, 4.12.3, 4

#Get QA code from the transformers library `transformers==4.3.3`

In [4]:
!git clone https://github.com/aub-mind/arabert
!cp arabert/examples/question-answering/utils_qa.py .
!cp arabert/examples/question-answering/trainer_qa.py .
!cp arabert/examples/question-answering/run_qa.py .
!cp arabert/examples/question-answering/squad_preprocessing.py .

Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 24.57 MiB/s, done.
Resolving deltas: 100% (339/339), done.


Download the TyDiQA data

In [5]:
!wget https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-train.json
!wget https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-dev.json

--2023-07-02 19:27:08--  https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-train.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.128, 142.250.148.128, 209.85.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58004076 (55M) [application/json]
Saving to: ‘tydiqa-goldp-v1.1-train.json’

tydiqa-goldp-v1.1-t 100%[===================>]  55.32M   129MB/s    in 0.4s    

2023-07-02 19:27:08 (129 MB/s) - ‘tydiqa-goldp-v1.1-train.json’ saved [58004076/58004076]

--2023-07-02 19:27:09--  https://storage.googleapis.com/tydiqa/v1.1/tydiqa-goldp-v1.1-dev.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.136.128, 142.250.148.128, 209.85.200.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.136.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5617409 (5.4M) [applicati

#We need to preprocess the files before training

Preprocessing using fuzzy search is a very slow process. I advise that you save the preprocessed files somewhere for later use.

In [6]:
model_name="aubmindlab/bert-base-arabertv02"

In [7]:
!rm -rf *-pre.json
!python squad_preprocessing.py \
  --input_file "tydiqa-goldp-v1.1-train.json" \
  --output_file "tydiqa-goldp-v1.1-train-pre.json" \
  --model_name=$model_name \
  --filter_tydiqa=True

2023-07-02 19:27:09.895668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 19:27:10.894941: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-02 19:27:12.747405: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-02 19:27:13.294001: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [7]:
!python squad_preprocessing.py \
  --input_file "tydiqa-goldp-v1.1-dev.json" \
  --output_file "tydiqa-goldp-v1.1-dev-pre.json" \
  --model_name=$model_name \
  --filter_tydiqa=True

2023-07-02 15:14:30.362632: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-02 15:14:31.315925: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-07-02 15:14:32.923552: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-07-02 15:14:32.982842: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

#Construct a custom dataset loader:


In [8]:
#@title
%%writefile artydiqa.py
"""TODO(tydiqa): Add a description here."""

from __future__ import absolute_import, division, print_function

import json
import textwrap

import datasets


# TODO(tydiqa): BibTeX citation
_CITATION = """\
@article{tydiqa,
title   = {TyDi QA: A Benchmark for Information-Seeking Question Answering in Typologically Diverse Languages},
author  = {Jonathan H. Clark and Eunsol Choi and Michael Collins and Dan Garrette and Tom Kwiatkowski and Vitaly Nikolaev and Jennimaria Palomaki}
year    = {2020},
journal = {Transactions of the Association for Computational Linguistics}
}
"""

# TODO(tydiqa):
_DESCRIPTION = """\
TyDi QA is a question answering dataset covering 11 typologically diverse languages with 204K question-answer pairs.
The languages of TyDi QA are diverse with regard to their typology -- the set of linguistic features that each language
expresses -- such that we expect models performing well on this set to generalize across a large number of the languages
in the world. It contains language phenomena that would not be found in English-only corpora. To provide a realistic
information-seeking task and avoid priming effects, questions are written by people who want to know the answer, but
don’t know the answer yet, (unlike SQuAD and its descendents) and the data is collected directly in each language without
the use of translation (unlike MLQA and XQuAD).
"""

# _URL = "https://storage.googleapis.com/tydiqa/"

# _PRIMARY_URLS = {
#     "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
#     "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
# }
# _SECONDARY_URLS = {
#     "train": _URL + "v1.1/tydiqa-goldp-v1.1-train.json",
#     "dev": _URL + "v1.1/tydiqa-goldp-v1.1-dev.json",
# }

#use this for AraBERTv1 and V2
_URL = "https://storage.googleapis.com/tydiqa/"
_URL2 = "/content/"
_PRIMARY_URLS = {
    "train": _URL + "v1.0/tydiqa-v1.0-train.jsonl.gz",
    "dev": _URL + "v1.0/tydiqa-v1.0-dev.jsonl.gz",
}
_SECONDARY_URLS = {
    "train": _URL2 + "tydiqa-goldp-v1.1-train-pre.json",
    "dev": _URL2 + "tydiqa-goldp-v1.1-dev-pre.json",
}


class TydiqaConfig(datasets.BuilderConfig):

    """ BuilderConfig for Tydiqa"""

    def __init__(self, **kwargs):
        """

        Args:
            **kwargs: keyword arguments forwarded to super.
        """
        super(TydiqaConfig, self).__init__(version=datasets.Version("1.0.0", ""), **kwargs)


class Tydiqa(datasets.GeneratorBasedBuilder):
    """TODO(tydiqa): Short description of my dataset."""

    # TODO(tydiqa): Set up version.
    VERSION = datasets.Version("0.1.0")
    BUILDER_CONFIGS = [
        TydiqaConfig(
            name="primary_task",
            description=textwrap.dedent(
                """\
          Passage selection task (SelectP): Given a list of the passages in the article, return either (a) the index of
          the passage that answers the question or (b) NULL if no such passage exists.
          Minimal answer span task (MinSpan): Given the full text of an article, return one of (a) the start and end
          byte indices of the minimal span that completely answers the question; (b) YES or NO if the question requires
          a yes/no answer and we can draw a conclusion from the passage; (c) NULL if it is not possible to produce a
          minimal answer for this question."""
            ),
        ),
        TydiqaConfig(
            name="secondary_task",
            description=textwrap.dedent(
                """Gold passage task (GoldP): Given a passage that is guaranteed to contain the
          answer, predict the single contiguous span of characters that answers the question. This is more similar to
          existing reading comprehension datasets (as opposed to the information-seeking task outlined above).
          This task is constructed with two goals in mind: (1) more directly comparing with prior work and (2) providing
          a simplified way for researchers to use TyDi QA by providing compatibility with existing code for SQuAD 1.1,
          XQuAD, and MLQA. Toward these goals, the gold passage task differs from the primary task in several ways:
          only the gold answer passage is provided rather than the entire Wikipedia article;
          unanswerable questions have been discarded, similar to MLQA and XQuAD;
          we evaluate with the SQuAD 1.1 metrics like XQuAD; and
         Thai and Japanese are removed since the lack of whitespace breaks some tools.
          """
            ),
        ),
    ]

    def _info(self):
        # TODO(tydiqa): Specifies the datasets.DatasetInfo object
        if self.config.name == "primary_task":
            return datasets.DatasetInfo(
                # This is the description that will appear on the datasets page.
                description=_DESCRIPTION,
                # datasets.features.FeatureConnectors
                features=datasets.Features(
                    {
                        "passage_answer_candidates": datasets.features.Sequence(
                            {
                                "plaintext_start_byte": datasets.Value("int32"),
                                "plaintext_end_byte": datasets.Value("int32"),
                            }
                        ),
                        "question_text": datasets.Value("string"),
                        "document_title": datasets.Value("string"),
                        "language": datasets.Value("string"),
                        "annotations": datasets.features.Sequence(
                            {
                                # 'annotation_id': datasets.Value('variant'),
                                "passage_answer_candidate_index": datasets.Value("int32"),
                                "minimal_answers_start_byte": datasets.Value("int32"),
                                "minimal_answers_end_byte": datasets.Value("int32"),
                                "yes_no_answer": datasets.Value("string"),
                            }
                        ),
                        "document_plaintext": datasets.Value("string"),
                        # 'example_id': datasets.Value('variant'),
                        "document_url": datasets.Value("string")
                        # These are the features of your dataset like images, labels ...
                    }
                ),
                # If there's a common (input, target) tuple from the features,
                # specify them here. They'll be used if as_supervised=True in
                # builder.as_dataset.
                supervised_keys=None,
                # Homepage of the dataset for documentation
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )
        elif self.config.name == "secondary_task":
            return datasets.DatasetInfo(
                description=_DESCRIPTION,
                features=datasets.Features(
                    {
                        "id": datasets.Value("string"),
                        "title": datasets.Value("string"),
                        "context": datasets.Value("string"),
                        "question": datasets.Value("string"),
                        "answers": datasets.features.Sequence(
                            {
                                "text": datasets.Value("string"),
                                "answer_start": datasets.Value("int32"),
                            }
                        ),
                    }
                ),
                # No default supervised_keys (as we have to pass both question
                # and context as input).
                supervised_keys=None,
                homepage="https://github.com/google-research-datasets/tydiqa",
                citation=_CITATION,
            )

    def _split_generators(self, dl_manager):
        """Returns SplitGenerators."""
        # TODO(tydiqa): Downloads the data and defines the splits
        # dl_manager is a datasets.download.DownloadManager that can be used to
        # download and extract URLs
        primary_downloaded = dl_manager.download_and_extract(_PRIMARY_URLS)
        secondary_downloaded = dl_manager.download_and_extract(_SECONDARY_URLS)
        if self.config.name == "primary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": primary_downloaded["dev"]},
                ),
            ]
        elif self.config.name == "secondary_task":
            return [
                datasets.SplitGenerator(
                    name=datasets.Split.TRAIN,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["train"]},
                ),
                datasets.SplitGenerator(
                    name=datasets.Split.VALIDATION,
                    # These kwargs will be passed to _generate_examples
                    gen_kwargs={"filepath": secondary_downloaded["dev"]},
                ),
            ]

    def _generate_examples(self, filepath):
        """Yields examples."""
        # TODO(tydiqa): Yields (key, example) tuples from the dataset
        if self.config.name == "primary_task":
            with open(filepath, encoding="utf-8") as f:
                for id_, row in enumerate(f):
                    data = json.loads(row)
                    passages = data["passage_answer_candidates"]
                    end_byte = [passage["plaintext_end_byte"] for passage in passages]
                    start_byte = [passage["plaintext_start_byte"] for passage in passages]
                    title = data["document_title"]
                    lang = data["language"]
                    question = data["question_text"]
                    annotations = data["annotations"]
                    # annot_ids = [annotation["annotation_id"] for annotation in annotations]
                    yes_no_answers = [annotation["yes_no_answer"] for annotation in annotations]
                    min_answers_end_byte = [
                        annotation["minimal_answer"]["plaintext_end_byte"] for annotation in annotations
                    ]
                    min_answers_start_byte = [
                        annotation["minimal_answer"]["plaintext_start_byte"] for annotation in annotations
                    ]
                    passage_cand_answers = [
                        annotation["passage_answer"]["candidate_index"] for annotation in annotations
                    ]
                    doc = data["document_plaintext"]
                    # example_id = data["example_id"]
                    url = data["document_url"]
                    yield id_, {
                        "passage_answer_candidates": {
                            "plaintext_start_byte": start_byte,
                            "plaintext_end_byte": end_byte,
                        },
                        "question_text": question,
                        "document_title": title,
                        "language": lang,
                        "annotations": {
                            # 'annotation_id': annot_ids,
                            "passage_answer_candidate_index": passage_cand_answers,
                            "minimal_answers_start_byte": min_answers_start_byte,
                            "minimal_answers_end_byte": min_answers_end_byte,
                            "yes_no_answer": yes_no_answers,
                        },
                        "document_plaintext": doc,
                        # 'example_id': example_id,
                        "document_url": url,
                    }
        elif self.config.name == "secondary_task":
            with open(filepath, encoding="utf-8") as f:
                data = json.load(f)
                for article in data["data"]:
                    title = article.get("title", "").strip()
                    for paragraph in article["paragraphs"]:
                        context = paragraph["context"].strip()
                        for qa in paragraph["qas"]:
                            question = qa["question"].strip()
                            id_ = qa["id"]
                            if "arabic" not in id_:
                              continue
                            answer_starts = [answer["answer_start"] for answer in qa["answers"] if answer["answer_start"] != -1]
                            answers = [answer["text"].strip() for answer in qa["answers"] if answer["answer_start"] != -1]
                            if len(answers) == 0 or len(answer_starts)==0:
                              print("question skipped")
                              continue
                            assert len(answer_starts)==len(answers)
                            # Features currently used are "context", "question", and "answers".
                            # Others are extracted here for the ease of future expansions.
                            yield id_, {
                                "title": title,
                                "context": context,
                                "question": question,
                                "id": id_,
                                "answers": {
                                    "answer_start": answer_starts,
                                    "text": answers,
                                },
                            }

Writing artydiqa.py


#Run Pytorch Training

In [ ]:
#!python run_qa.py --help

In [9]:
model_name="aubmindlab/araelectra-base-discriminator"

In [10]:
!python run_qa.py \
  --model_name_or_path $model_name \
  --dataset_name artydiqa.py \
  --dataset_config_name "secondary_task" \
  --do_train \
  --do_eval \
  --per_device_train_batch_size 8 \
  --gradient_accumulation_steps 1 \
  --learning_rate 3e-5 \
  --num_train_epochs 4 \
  --max_seq_length 384 \
  --doc_stride 128 \
  --output_dir ./run \
  --n_best_size 20 \
  --evaluation_strategy epoch \
  --save_steps 20160 \
  --overwrite_output_dir \
  --seed 42 \
  --warmup_steps 500 \
  # --load_best_model_at_end \
  # --metric_for_best_model exact_match \
  # --greater_is_better True

Traceback (most recent call last):
  File "/content/run_qa.py", line 29, in <module>
    import transformers
ModuleNotFoundError: No module named 'transformers'


#Run Inference using the ready model


In [3]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 86.8 MB/s eta 0:00:00


In [4]:
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic

Cloning into 'arabert'...
remote: Enumerating objects: 600, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 600 (delta 38), reused 45 (delta 30), pack-reused 535
Receiving objects: 100% (600/600), 9.14 MiB | 13.59 MiB/s, done.
Resolving deltas: 100% (339/339), done.


In [26]:
from arabert.preprocess import ArabertPreprocessor
from transformers import pipeline

prep = ArabertPreprocessor("aubmindlab/araelectra-base-discriminator") #or empty string it's the same
qa_pipe_araelectra =pipeline("question-answering",model="wissamantoun/araelectra-base-artydiqa")



In [50]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/xlm-roberta-large-squad2"

# a) Get predictions
qa_pipe_xlm = pipeline('question-answering', model=model_name, tokenizer=model_name)

In [33]:


context1 = """
تنظيم إجراءات المنافسات والمشتريات التي تقوم بها الجهات الحكومية ومنع تأثير المصالح الشخصية فيها، وذلك حماية للمال العام و تحقيق أقصى درجات الكفاية الاقتصادية للحصول على المشتريات الحكومية وتنفيذ مشروعاتها بأسعار تنافسية عادلة و تعزيز النزاهة والمنافسة، وتوفير معاملة عادلة للمتعهدين والمقاولين؛ تحقيقًا لمبدأ تكافؤ الفرص .و تحقيق الشفافية في جميع مراحل إجراءات المنافسات والمشتريات الحكومية .
المادة الثانية، تتعامل الجهات الحكومية عند تنفيذ منافساتها وتوفير مشترياتها مع الأفراد والمؤسسات والشركات المرخص لهم بمزاولة العمل الذي تقع في نطاقه الأعمال طبقًا للأنظمة والقواعد المتبعة.
المادة الثالثة ،مع مراعاة ما ورد في نظام الاستثمار الأجنبي ، يعطى جميع الأفراد والمؤسسات والشركات الراغبين في التعامل مع الحكومة ممن تتوافر فيهم الشروط التي تؤهلهم لهذا التعامل فرصًا متساوية ويعاملون على قدم المساواة .
المادة الرابعة ، توفر للمتنافسين المعلومات الواضحة الكاملة والموحدة عن العمل المطلوب ، ويمكنون من الحصول على هذه المعلومات في وقت محدد ، كما توفر نسخ كافية من وثائق المنافسة لتلبية طلبات الراغبين في الحصول عليها .
المادة الخامسة، تكون الأولوية في التعامل للمصنوعات والمنتجات والخدمات الوطنية وما يعامل معاملتها .
المادة السادسة، تطرح جميع الأعمال والمشتريات الحكومية في منافسة عامة عدا ما يستثنى من المنافسة بموجب أحكام هذا النظام .
المادة السابعة، يعلن عن جميع المنافسات الحكومية في الجريدة الرسمية وفي صحيفتين محليتين ، وبالوسائل الإعلانية الإلكترونية وفقًا لما تحدده اللائحة التنفيذية لهذا النظام ، ويجب أن يحدد في الإعلان عن المنافسة موعد تقديم العروض وفتح المظاريف ومكانهما . الأعمال أو المشاريع ذات الطبيعة الخاصة التي لا يتوفر لها متعهد أو المادة مقاول داخل المملكة يتم الإعلان عنها خارج المملكة بالإضافة إلى الإعلان عنها في الداخل وفقًا لما تضمنته الفقرة السابقة .
المادة الثامنة لا يجوز قبول العروض والتعاقد بموجبها إلا طبقًا للشروط والمواصفات الموضوعة لها .
المادة التاسعة ،يجب أن يتم الشراء وتنفيذ الأعمال والمشاريع بأسعار عادلة لا تزيد على الأسعار السائدة ، وتعد المنافسة الوسيلة العملية للوصول إلى ذلك وفق الأحكام الواردة في هذا النظام .
المادة العاشرة، تقدم العروض في مظاريف مختومة في الموعد والمكان المحددين لقبولها . ولا يجوز قبول العروض التي تقدم أو تصل إلى الجهة الحكومية بعد انتهاء الموعد المحدد لتقديمها. ويجوز تقديم العروض وفتحها عن طريق الوسائل الإلكترونية وفقًا لما تحدده اللائحة التنفيذية لهذا النظام ، وتعلن الجهة الحكومية عن أسماء الشركات والمؤسسات التي تقدمت بعروضها .
المادة الحادية عشرة، يقدم مع العرض ضمان ابتدائي يتراوح من (1%) إلى (2%) (من واحد إلى اثنين في المائة) من قيمته وفقًا لشروط المنافسة ، ولا يلزم تقديم هذا الضمان في الحالات التالية :  الشراء المباشر (إلا إذا كانت العروض مغلقة) .  تعاقدات الجهات الخاضعة لأحكام هذا النظام فيما بينها ، وفي التعاقد مع الجمعيات الخيرية والجمعيات ذات النفع العام ، بشرط تنفيذها الأعمال بنفسها .
المادة الثانية عشرة،  تكون مدة سريان العروض في المنافسات العامة تسعين يومًا من التاريخ المحدد لفتح المظاريف ، فإن سحب مقدِّم العرض عرضه قبل انتهاء هذه المدة فلا يعاد له ضمانه الابتدائي .
 لا يجوز تمديد مدة سريان العرض والضمان الابتدائي إلا بموافقة مقدِّم العرض.
المادة الثالثة عشرة ، يجب أن تحدد الأسعار الإجمالية وما يرد عليها من زيادة أو تخفيض في خطاب العرض الأصلي . ولا يعتد بأي تخفيض يقدم بوساطة خطاب مستقل حتى لو كان مرافقًا للعرض .
ولا يجوز للمتنافسين في غير الحالات التي يجوز التفاوض فيها وفقًا لأحكام هذا النظام تعديل أسعار عروضهم بالزيادة أو التخفيض بعد تقديمها .
المادة الرابعة عشرة ،تكوّن لجنة أو أكثر لدى الجهة الحكومية لفتح المظاريف لا يقل عدد أعضائها عن ثلاثة إضافة إلى رئيسها الذي لا تقل مرتبته عن العاشرة أو ما يعادلها . وينص في التكوّين على عضو احتياطي يكمل النصاب إن غاب أحد الأعضاء . ويعاد تكوّين اللجنة كل ثلاث سنوات .
المادة الخامسة عشرة ،تفتح المظاريف بحضور جميع أعضاء لجنة فتح المظاريف في الموعد المحدد لذلك . وتعلن على من حضر من المتنافسين أو مندوبيهم الأسعار الواردة في العروض ، ويجب على هذه اللجنة إحالة محضرها وأوراق المنافسة إلى لجنة فحص العروض خلال سبعة أيام من تاريخ فتح المظاريف .
المادة السادسة عشرة ،  تكوّن في الجهة الحكومية لجنة أو أكثر لفحص العروض تتكون من ثلاثة أعضاء على الأقل إضافة إلى رئيسها الذي لا تقل مرتبته عن الثالثة عشرة أو ما يعادلها ، على أن يكون من بينهم المراقب المالي ومن هو مؤهل تأهيلًا نظاميًّا . ويُنص في التكوّين على عضو احتياطي يكمل النصاب إن غاب أحد الأعضاء ، وتتولى هذه اللجنة تقديم توصياتها في الترسية على أفضل العروض وفقًا لأحكام هذا النظام ولائحته التنفيذية ، ولها أن تستعين في تقديم توصياتها بتقرير من فنيين متخصصين .
 يعاد تكوين اللجنة كل سنة .
المادة السابعة عشرة ،لا يجوز الجمع بين رئاسة لجنة فحص العروض وصلاحية البت في المنافسة ، كما لا يجوز الجمع بين رئاسة لجنة فتح المظاريف ورئاسة لجنة فحص العروض أو العضوية فيهما .
المادة الثامنة عشرة ، يجوز أن يرأس لجنة فحص العروض موظف لا تقل مرتبته عن العاشرة أو ما يعادلها ، إذا كانت مُشكَّلة في غير مقر الجهة الرئيس .
المادة التاسعة عشرة ، تتخذ اللجنة توصياتها بحضور كامل أعضائها ، وتدون هذه التوصيات في محضر ، ويوضح الرأي المخالف إن وجد ، وحجة كلا الرأيين ، ليعرض على صاحب الصلاحية للبت في الترسية بما يتفق مع أحكام هذا النظام .
المادة العشرون ، يجب على الجهة الحكومية البت في العروض واعتماد الترسية خلال المدة المحددة لسريان العروض المشار إليها في المادة الثانية عشرة ، وبعد انتهاء هذه المدة تتخذ الإجراءات اللازمة لإعادة الضمانات الابتدائية لأصحابها .
المادة الحادية والعشرون ،  يجوز للجنة فحص العروض التفاوض مع صاحب أقل عرض مطابق للشروط والمواصفات ثم مع من يليه من المتنافسين في الحالتين التاليتين
إذا ارتفعت العروض عن أسعار السوق بشكل ظاهر تحدد اللجنة مبلغ التخفيض بما يتفق مع أسعار السوق ، وتطلب كتابيًّا من صاحب العرض الأقل تخفيض سعره . فإن امتنع أو لم يصل بسعره إلى المبلغ المحدد ، تتفاوض مع العرض الذي يليه وهكذا . فإن لم يتم التوصل إلى السعر المحدد تلغى المنافسة ، ويعاد طرحها من جديد .  إذا زادت قيمة العروض على المبالغ المعتمدة للمشروع ، يجوز للجهة الحكومية إلغاء بعض البنود أو تخفيضها للوصول إلى المبالغ المعتمدة بشرط أن لا يؤثر ذلك على الانتفاع بالمشروع أو ترتيب العروض وإلا تُلغى المنافسة .
المادة الثانية والعشرون ، لا يجوز استبعاد أي عرض بحجة تدني أسعاره إلا إذا قل بنسبة (35%) خمسة وثلاثين في المائة فأكثر عن تقديرات الجهة الحكومية والأسعار السائدة ، ويجوز للجنة فحص العروض بعد مناقشة صاحب العرض وإجراء التحليل المالي والفني ووصولها إلى قناعة بمقدرة صاحب العرض على تنفيذ العقد التوصية بعدم استبعاد العرض .
المادة الثالثة والعشرون ، يجوز للجنة فحص العروض التوصية باستبعاد أي عرض من العروض من المنافسة حتى لو كان أقل العروض سعرًا ، إذا تبين أن لدى صاحب العرض عددًا من المشاريع ورأت اللجنة أن حجم التزاماته التعاقدية قد أصبح مرتفعًا على نحو يفوق قدراته المالية أو الفنية بما يؤثر على تنفيذه لالتزاماته التعاقدية ، وفي هذه الحالة تتفاوض مع العطاء الذي يليه وفقًا لقواعد التفاوض المحددة في هذا النظام .
المادة الرابعة والعشرون ،إذا لم يقدم للمنافسة إلا عرض واحد ، أو قدمت عدة عروض واتضح أنها غير مطابقة للشروط والمواصفات - عدا عرض واحد - فلا يجوز قبول هذا العرض إلا إذا كانت أسعاره مماثلة للأسعار السائدة ، وكانت حاجة العمل لا تسمح بإعادة طرح المنافسة مرة أخرى ، وذلك بعد موافقة الوزير المختص أو رئيس الدائرة المستقلة .
المادة الخامسة والعشرون،مع مراعاة ما ورد في المادتين (الحادية والعشرين) و(الرابعة والعشرين) من هذا النظام،
لا يجوز إلغاء المنافسة إلا للمصلحة العامة ، أو لمخالفة إجراءاتها أحكام النظام ، أو لوجود أخطاء جوهرية مؤثرة في الشروط أو المواصفات ، وتكون صلاحية الإلغاء للوزير المختص أو رئيس الدائرة المستقلة .
ترد لأصحاب العروض قيمة وثائق المنافسة في حالة الإلغاء لأسباب تعود للجهة الحكومية .
"""
context2=""" المادة الخامسة والثلاثون، توضح اللائحة التنفيذية لهذا النظام شروط الضمانات البنكية والمالية وأحكامها ونماذجها .
المادة السادسة والثلاثون،يجوز للجهة الحكومية زيادة التزامات المتعاقد ضمن نطاق العقد بما لا يتجاوز (10%) عشرة في المائة من القيمة الإجمالية للعقد ، أو تخفيض هذه الالتزامات بما لا يتجاوز (20%) عشرين في المائة . وتوضح اللائحة التنفيذية الضوابط اللازمة لذلك.
المادة السابعة والثلاثون، تدفع قيمة العقود بالريال السعودي . ويجوز أن تدفع بأي عملة أخرى بعد التنسيق مع وزارة المالية . وينص في شروط المنافسة على العملة التي يقدم بها العرض ، على أنه لا يجوز دفع قيمة العقد بأكثر من عملة واحدة .
المادة الثامنة والثلاثون، يجوز للجهة الحكومية أن تدفع للمتعاقد معها دفعة مقدمة من استحقاقه بنسبة (5%) خمسة في المائة من القيمة الإجمالية للعقد ، بشرط ألا تتجاوز قيمة الدفعة مبلغ خمسين مليون ريال أو ما يعادلها مقابل ضمان بنكي مساوٍ لهذه القيمة ، وينص على الدفعة المقدمة ، إن وجدت ، في الشروط والمواصفات عند طرح المنافسة ، وتحسم هذه الدفعة من مستخلصات المتعاقد على أقساط ابتداءً من المستخلص الأول ، وفقًا للضوابط الموضحة في اللائحة التنفيذية .
المادة التاسعة والثلاثون، تصرف مستحقات المقاول على دفعات طبقًا لما يتم إنجازه من عمل بموجب المستخلصات التي تعتمدها الجهة الحكومية .
المادة الأربعون،يصرف المستخلص الأخير الذي يجب ألا يقل عن (10%) عشرة في المائة في عقود الأشغال العامة وعن (5%) خمسة في المائة في العقود الأخرى بعد تسليم الأعمال تسليمًا ابتدائيًّا ، أو توريد المشتريات .
المادة الحادية والأربعون،يجوز للجهة الحكومية عند الحاجة وبعد الاتفاق مع وزارة المالية أن تنفذ بعض مشاريعها بحيث تسدد التكاليف على أقساط سنوية على أن تطرح مثل تلك الأعمال في منافسة عامة .
المادة الثانية والأربعون،تكون القيمة الإجمالية للعقد شاملة ومغطية جميع تكاليف تنفيذه وفقًا لشروطه ، بما في ذلك قيمة الرسوم والضرائب التي يدفعها المتعاقد . ولا يجوز الإعفاء منها أو إعفاء أرباح المتعاقدين مع الجهة الحكومية أو دُخول موظفيهم من الضريبة أو دفعها عنهم عدا ما استثني من ذلك بموجب نص نظامي خاص .
 """


context3="""  المادة السابعة عشرة ،لا يجوز الجمع بين رئاسة لجنة فحص العروض وصلاحية البت في المنافسة ، كما لا يجوز الجمع بين رئاسة لجنة فتح المظاريف ورئاسة لجنة فحص العروض أو العضوية فيهما .
المادة الثامنة عشرة ، يجوز أن يرأس لجنة فحص العروض موظف لا تقل مرتبته عن العاشرة أو ما يعادلها ، إذا كانت مُشكَّلة في غير مقر الجهة الرئيس .  """



In [48]:
text =  "ما هي الأولوية في المعاملات؟"
context = prep.preprocess(context1)# don't forget to preprocess the question and the context to get the optimal results

result = qa_pipe_araelectra(question=text,context=context)
QA_input = {'question':text,'context': context}

res = qa_pipe_xlm(QA_input)

IndexError: ignored

In [49]:


result

{'score': 0.528122067451477,
 'start': 1062,
 'end': 1117,
 'answer': 'للمصنوعات والمنتجات والخدمات الوطنية وما يعامل معاملتها'}

In [42]:
res

{'score': 0.7157438397407532,
 'start': 1061,
 'end': 1098,
 'answer': ' للمصنوعات والمنتجات والخدمات الوطنية'}